# Download Dataset

In [1]:
%%bash
pip install kagglehub
pip install opencv-python-headless
pip install scikit-learn


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [10]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("imadeddinedjerarda/mrl-eye-dataset")

print("Path to dataset files:", path)

100%|██████████| 330M/330M [00:49<00:00, 7.02MB/s] 

Extracting files...


Path to dataset files: /Users/atziripe/.cache/kagglehub/datasets/imadeddinedjerarda/mrl-eye-dataset/versions/1


In [2]:

%%bash
cd  /Users/atziripe/.cache/kagglehub/datasets/imadeddinedjerarda/mrl-eye-dataset/versions/1
ls

bash: line 1: cd: /Users/atziripe/.cache/kagglehub/datasets/imadeddinedjerarda/mrl-eye-dataset/versions/1: No such file or directory


dataset
preprocessing.ipynb
resized_dataset


In [12]:
%%bash
mkdir dataset/
mv /Users/atziripe/.cache/kagglehub/datasets/imadeddinedjerarda/mrl-eye-dataset/versions/1 dataset/

In [25]:
#check size of resized image and resize it if needed for the model
import cv2

image = cv2.imread('resized_dataset/Close-Eyes/s0001_00001_0_0_0_0_0_01.png')

# Get the dimensions of the image
height, width, channels = image.shape

print(f"Width: {width}, Height: {height}, Channels: {channels}")

Width: 64, Height: 64, Channels: 3


In [24]:
from PIL import Image
import os
import numpy as np

def preprocess_images(input_dir, output_dir, target_size=(64, 64)):
    for class_name in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_name)
        
        if os.path.isdir(class_path):
            print(f"Processing class: {class_name}")
            
            output_class_dir = os.path.join(output_dir, class_name)
            os.makedirs(output_class_dir, exist_ok=True)

            for img_name in os.listdir(class_path):
                img_path = os.path.join(class_path, img_name)
                try:
                    img = Image.open(img_path)
                    img = img.convert("L")  # "L" gray scale in PIL
                    img = img.resize(target_size)
                    img.save(os.path.join(output_class_dir, img_name), format='PNG') 
                except Exception as e:
                    print(f"Error procesando {img_path}: {e}")

input_dir = "dataset/1/mrleyedataset/"
output_dir = "resized_dataset/"

preprocess_images(input_dir, output_dir, target_size=(64, 64))


Procesando clase: Close-Eyes
Procesando clase: Open-Eyes


In [26]:
def split_dataset(input_dir, output_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    for class_name in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_name)
        if os.path.isdir(class_path):
            images = [os.path.join(class_path, img) for img in os.listdir(class_path) if img.endswith(('.jpg', '.png'))]

            train, temp = train_test_split(images, test_size=(1 - train_ratio), random_state=42)
            val, test = train_test_split(temp, test_size=(test_ratio / (val_ratio + test_ratio)), random_state=42)

            for split_name, split_data in zip(['train', 'validation', 'test'], [train, val, test]):
                split_dir = os.path.join(output_dir, split_name, class_name)
                os.makedirs(split_dir, exist_ok=True)

                for img_path in split_data:
                    shutil.copy(img_path, split_dir)

split_dataset("resized_dataset", "final_dataset")

In [27]:
#Confirm new dimensions
image = cv2.imread('final_dataset/train/Close-Eyes/s0001_00001_0_0_0_0_0_01.png')

# Get the dimensions of the image
height, width, channels = image.shape

print(f"Width: {width}, Height: {height}, Channels: {channels}")

Width: 64, Height: 64, Channels: 3
